In [2]:
import click
import numpy as np
import logging
import pandas as pd

from constants import DIRECTORY

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


df = pd.read_parquet(f"{DIRECTORY}/test.parquet")

df.head()

,query_id,query,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,gain
16,2,!qscreen fence without holes,B07D7TBSGH,FOTMISHU 6Pcs Greenhouse Hoops Rust-Free Grow ...,<p><b>Description:</b><br><b>Material:</b>plas...,►Material -- High-quality plastic coated steel...,FOTMISHU,green,0.0
17,2,!qscreen fence without holes,B07DHT2WZK,Zippity Outdoor Products ZP19028 Unassembled M...,None,Designed for the Madison Vinyl Fence (item ZP1...,Zippity Outdoor Products,White,0.0
18,2,!qscreen fence without holes,B07DHX8YH2,Zippity Outdoor Products ZP19026 Lightweight P...,None,Designed as a portable fence for temporary eve...,Zippity Outdoor Products,White,1.0
19,2,!qscreen fence without holes,B07DS1YCRZ,ColourTree 4' x 50' Green Fence Privacy Screen...,None,"★【Virgin, UV Stabilized Compounds & Thick Mate...",ColourTree,Green 2nd Generation,0.1
20,2,!qscreen fence without holes,B07DS3J3MB,ColourTree 6' x 50' Black Fence Privacy Screen...,None,"★【Virgin, UV Stabilized Compounds & Thick Mate...",ColourTree,Black 2nd Generation,0.1


In [57]:
cols = ["query", "product_brand", "product_title"]
df_c = df[(df.gain == 1) & (df.product_brand.notnull())][cols].sample(1000).copy()
df_c = df_c.applymap(lambda x: x.lower() if isinstance(x, str) else x)
logging.info(df_c.shape)
df_c.sample(5)[cols[:-1]]

INFO:root:(1000, 3)


,query,product_brand
63289,adhesive towel bar 3m,ulifestar
551843,under armour volleyball spandex,under armour
375356,molle gear,gear aid
498410,soccer cleats mens,adidas
93938,banana popsicles,davinci gourmet


In [60]:
import re


def get_full_matches(row, entity_type="product_brand"):
    entity = row[entity_type]
    query = row["query"]
    present = re.search(r"\b{}\b".format(re.escape(entity)), query)
    if present:
        start = present.start()
        end = present.end()
        start_token = query[:start].count(" ")
        end_token = query[:end].count(" ")
        return {
            "tokenized_text": query.split(" "),
            "ner": [start_token, end_token, entity_type],
        }


df_c["entities"] = df_c.apply(get_full_matches, axis=1)

df_c = df_c[df_c.entities.notnull()]
df_c

,query,product_brand,product_title,entities
369087,midol long lasting relief,midol,"midol complete caplets, 24-count (pack of 2), ...","{'tokenized_text': ['midol', 'long', 'lasting'..."
542940,trailer brake controller curt,curt,"curt 51170 spectrum original equipment style, ...","{'tokenized_text': ['trailer', 'brake', 'contr..."
138927,camelbak bite vavle kids,camelbak,"camelbak eddy kids bite valves, 4-pack","{'tokenized_text': ['camelbak', 'bite', 'vavle..."
480352,sesame street sticks,sesame street,sesame street first books series; big bird's l...,"{'tokenized_text': ['sesame', 'street', 'stick..."
551843,under armour volleyball spandex,under armour,under armour women's heatgear armour shorty sh...,"{'tokenized_text': ['under', 'armour', 'volley..."
...,...,...,...,...
551842,under armour volleyball spandex,under armour,"under armour women's on the court 3"" shorts,bl...","{'tokenized_text': ['under', 'armour', 'volley..."
151944,chanel lipstick long lasting,chanel,rouge coco shine hydrating sheer lipshine - # ...,"{'tokenized_text': ['chanel', 'lipstick', 'lon..."
274797,hello kitty figurine mini,hello kitty,hello kitty crystal kitty doll - limited edition,"{'tokenized_text': ['hello', 'kitty', 'figurin..."
419231,petzl escalada,petzl,"petzl - micro swivel, compact gated swivel","{'tokenized_text': ['petzl', 'escalada'], 'ner..."


In [61]:
df_c["entities"].to_json("data.json", orient="records")